### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random

In [3]:
import pandas as pd

# import gspread
# from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
# scope = ["https://spreadsheets.google.com/feeds"]
# credentials = ServiceAccountCredentials.from_json_keyfile_name(
#    "jupyter-integration-credentials.json", scope
# )
# gc = gspread.authorize(credentials)

In [7]:
# spreadsheet_key = "1_RXzXkHPEyDAiDKmz98wTSC9UNo-8OyjGTT540X_vJk"
# book = gc.open_by_key(spreadsheet_key)

In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [9]:
# Opening the worksheet by using Worksheet ID
# workbook = gc.open_by_key(spreadsheet_key)
# Selecting which sheet to pulling the data
# sheet = workbook.worksheet("Sheet1")
# Pulling the data and transform it to the data frame
# values = sheet.get_all_values()

#### read in opinion data

In [10]:
identity_df = pd.read_excel(
    "data/raw/tawian_publicopinion", sheet_name="data", skiprows=11, skipfooter=9
)

In [11]:
unification_df = pd.read_excel(
    "data/raw/tawian_publicopinion", sheet_name="data", skiprows=17
).drop(columns={"Unnamed: 30", "Unnamed: 31"})

#### clean up year columns

In [12]:
identity_df.columns = [
    "identity",
    "1992",
    "1993",
    "1994",
    "1995",
    "1996",
    "1997",
    "1998",
    "1999",
    "2000",
    "2001",
    "2002",
    "2003",
    "2004",
    "2005",
    "2006",
    "2007",
    "2008",
    "2009",
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

In [13]:
unification_df.columns = [
    "unification",
    "1994",
    "1995",
    "1996",
    "1997",
    "1998",
    "1999",
    "2000",
    "2001",
    "2002",
    "2003",
    "2004",
    "2005",
    "2006",
    "2007",
    "2008",
    "2009",
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

#### clean unification/independence data

In [14]:
unification_clean = unification_df.transpose()

In [15]:
header = unification_clean.iloc[0]

In [16]:
unification_clean = unification_clean[1:]

In [17]:
unification_clean.columns = header

In [18]:
unification_clean.columns = [
    "unification_asap",
    "maintain_unification",
    "maintain_decide_later",
    "maintain",
    "maintain_ind",
    "ind_asap",
    "no_response",
]

#### charting independence

In [19]:
unification_clean["lean_independence"] = (
    unification_clean["ind_asap"] + unification_clean["maintain_ind"]
)
unification_clean["lean_unification"] = (
    unification_clean["unification_asap"] + unification_clean["maintain_unification"]
)
unification_clean["undecided"] = (
    unification_clean["maintain"] + unification_clean["maintain_decide_later"]
)

In [20]:
unif_ind = unification_clean.reset_index().rename(columns={"index": "year"})

In [21]:
unif_ind_slim = unif_ind[["year", "lean_independence", "lean_unification", "undecided"]]

In [22]:
unif_ind_slim_long = pd.melt(
    unif_ind_slim,
    id_vars="year",
    value_vars=["lean_independence", "lean_unification", "undecided"],
)

In [23]:
alt.Chart(unif_ind_slim_long).mark_line(point=True).encode(
    x=alt.X("year:O"), y="value", color=alt.Color("variable:N")
)

alt.Chart(...)

In [24]:
alt.Chart(unif_ind_slim_long).mark_bar().encode(
    column=alt.Column("year:O", header=alt.Header(labelOrient="bottom")),
    x="variable:O",
    y="value",
    color="variable:N",
).properties(width=50)

alt.Chart(...)

#### now cleaning and charting the identity data

In [25]:
id_clean = identity_df.transpose()

In [26]:
header = id_clean.iloc[0]

In [27]:
id_clean = id_clean[1:]

In [28]:
id_clean.columns = header

In [29]:
id_clean = id_clean.reset_index()

In [30]:
id_clean.columns = ["year", "taiwanese", "both", "chinese", "no_response"]

In [31]:
bad_years = ("1992", "1993")
id_clean = id_clean[~id_clean["year"].isin(bad_years)].drop(columns=["no_response"])

In [32]:
id_long = pd.melt(id_clean, id_vars="year", value_vars=["taiwanese", "both", "chinese"])

In [33]:
alt.Chart(id_long).mark_line(point=True).encode(
    x="year:O", y="value", color="variable:N"
)

alt.Chart(...)

In [34]:
alt.Chart(id_long).mark_bar().encode(
    column=alt.Column("year:O", header=alt.Header(labelOrient="bottom")),
    x="variable:O",
    y="value",
    color="variable:N",
).properties(width=50)

alt.Chart(...)

#### put data for both in datawrapper

In [35]:
# dw.create_chart(chart_type="d3-lines")

In [36]:
dw.add_data(chart_id="c3lNJ", data=id_clean)

<Response [204]>

In [37]:
# dw.create_chart(chart_type="d3-lines", data=unif_ind_slim)

In [38]:
# dw.add_data(chart_id="n1vb0", data=unif_ind_slim)